In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

### создаем признаки для команды
- xp_600 тащера 1, опыт тащера 2, опыт керри, опыт остальных к 10 минуте игры (4 признака)
- gold_600 для тех же групп к 10 минуте (4 признака)
- lh_600 для всех групп (4 признака)

- внутри команды разделим героев на группы: 1 -- с наибольшим xp на каждую минуту игры (здесь будут два героя -- это одиночки) (можно раздробить на две группы: по убыванию опыта), 2 -- из оставшихся трех героев найдем героя с наибольшим xp (этот герой будет керри), 3 -- остальных двух 
- затем сделаем дамми на группы -- таким образом дадим модели понять, является ли герой самым крутым одиночкой, вторым самым крутым одиночкой, керри, или одним из двух героев кто поддерживает керри

In [2]:
df = pd.read_csv('main_train.csv')
tf = pd.read_csv('teamfights_train.csv')

In [3]:
df.head(10)

,match_id,radiant,hero,gold_0,lh_0,xp_0,gold_60,lh_60,xp_60,gold_120,...,xp_600,level_180,level_240,level_300,level_360,level_420,level_480,level_540,level_600,radiant_win
0,0,1,Rubick,0.0,0.0,0.0,100.0,0.0,46.0,250.0,...,1741.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0,1,Wraith King,0.0,0.0,0.0,175.0,2.0,124.0,526.0,...,2319.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1
2,0,1,Riki,0.0,0.0,0.0,137.0,1.0,93.0,439.0,...,3859.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1
3,0,1,Tusk,0.0,0.0,0.0,100.0,0.0,62.0,200.0,...,1676.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
4,0,1,Templar Assassin,0.0,0.0,0.0,320.0,3.0,352.0,668.0,...,4453.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1
5,0,0,Bounty Hunter,0.0,0.0,0.0,100.0,0.0,72.0,200.0,...,1644.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1
6,0,0,Invoker,0.0,0.0,0.0,143.0,1.0,155.0,362.0,...,2653.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
7,0,0,Earthshaker,0.0,0.0,0.0,100.0,0.0,103.0,255.0,...,1804.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8,0,0,Anti-Mage,0.0,0.0,0.0,178.0,2.0,103.0,522.0,...,3011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
9,0,0,Shadow Fiend,0.0,0.0,0.0,242.0,1.0,255.0,508.0,...,2965.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,1


In [12]:
len(df) / 10

39959.0

In [13]:
x = [0,1,2,3,4,128,129,130,131,132]
df['player_slot'] = x * 39959

In [15]:
df.head(10)

,match_id,radiant,hero,gold_0,lh_0,xp_0,gold_60,lh_60,xp_60,gold_120,...,level_180,level_240,level_300,level_360,level_420,level_480,level_540,level_600,radiant_win,player_slot
0,0,1,Rubick,0.0,0.0,0.0,100.0,0.0,46.0,250.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,0,1,Wraith King,0.0,0.0,0.0,175.0,2.0,124.0,526.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1,1
2,0,1,Riki,0.0,0.0,0.0,137.0,1.0,93.0,439.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1,2
3,0,1,Tusk,0.0,0.0,0.0,100.0,0.0,62.0,200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,3
4,0,1,Templar Assassin,0.0,0.0,0.0,320.0,3.0,352.0,668.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1,4
5,0,0,Bounty Hunter,0.0,0.0,0.0,100.0,0.0,72.0,200.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1,128
6,0,0,Invoker,0.0,0.0,0.0,143.0,1.0,155.0,362.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,129
7,0,0,Earthshaker,0.0,0.0,0.0,100.0,0.0,103.0,255.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,130
8,0,0,Anti-Mage,0.0,0.0,0.0,178.0,2.0,103.0,522.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,131
9,0,0,Shadow Fiend,0.0,0.0,0.0,242.0,1.0,255.0,508.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,1,132


In [4]:
tf.head(10)

,teamfight_id,match_id,player_slot,radiant,buybacks,damage,deaths_player,gold_delta,xp_end,xp_start,start,end,last_death
0,3288,1,0,1,0,673,0,89,2681,2263,423,467,452
1,3288,1,1,1,0,154,0,199,2016,1694,423,467,452
2,3288,1,2,1,0,403,0,320,1640,1230,423,467,452
3,3288,1,3,1,0,201,0,158,2367,1957,423,467,452
4,3288,1,4,1,0,369,1,165,2445,2061,423,467,452
5,3288,1,128,0,0,410,1,-149,1510,1479,423,467,452
6,3288,1,129,0,0,93,1,-119,1119,1088,423,467,452
7,3288,1,130,0,0,306,0,522,2055,1450,423,467,452
8,3288,1,131,0,0,0,0,190,3633,3215,423,467,452
9,3288,1,132,0,0,299,0,38,1367,1212,423,467,452


In [6]:
tf1 = tf.copy()
tf1['xp_delta'] = tf1['xp_end']-tf1['xp_start']
tf1['time'] = np.ceil(tf1['end'] / 60) * 60
tf2 = tf1[['teamfight_id','match_id','player_slot','radiant','damage','gold_delta','xp_delta','time']]

In [7]:
tf2.head(10)
#'teamfight_id','match_id','player_slot','radiant' -- для мерджа с df
# 'damage','gold_delta','xp_delta' -- разобьем на столбцы по time и добавим в df
# по командам без инфы надо добавлять нули

,teamfight_id,match_id,player_slot,radiant,damage,gold_delta,xp_delta,time
0,3288,1,0,1,673,89,418,480.0
1,3288,1,1,1,154,199,322,480.0
2,3288,1,2,1,403,320,410,480.0
3,3288,1,3,1,201,158,410,480.0
4,3288,1,4,1,369,165,384,480.0
5,3288,1,128,0,410,-149,31,480.0
6,3288,1,129,0,93,-119,31,480.0
7,3288,1,130,0,306,522,605,480.0
8,3288,1,131,0,0,190,418,480.0
9,3288,1,132,0,299,38,155,480.0


In [23]:
tf3 = tf2.groupby(['match_id','player_slot'],as_index=False)[['damage','gold_delta','xp_delta']].sum()

In [24]:
tf3.columns

Index(['match_id', 'player_slot', 'damage', 'gold_delta', 'xp_delta'], dtype='object')

In [25]:
tf3.head(10)

,match_id,player_slot,damage,gold_delta,xp_delta
0,1,0,769,274,621
1,1,1,944,548,715
2,1,2,403,444,709
3,1,3,386,248,482
4,1,4,816,-14,384
5,1,128,459,-98,188
6,1,129,186,-119,259
7,1,130,828,343,762
8,1,131,326,628,1013
9,1,132,359,172,383


In [26]:
tf3['match_id'].unique()

array([    1,     2,     3, ..., 39956, 39957, 39958], dtype=int64)

In [30]:
new_df = pd.merge(df, tf3,  how='left', left_on=['match_id','player_slot'], right_on = ['match_id','player_slot'])
new_df = new_df.fillna(0)

In [32]:
new_df.head()

,match_id,radiant,hero,gold_0,lh_0,xp_0,gold_60,lh_60,xp_60,gold_120,...,level_360,level_420,level_480,level_540,level_600,radiant_win,player_slot,damage,gold_delta,xp_delta
0,0,1,Rubick,0.0,0.0,0.0,100.0,0.0,46.0,250.0,...,0.0,0.0,0.0,0.0,0.0,1,0,0.0,0.0,0.0
1,0,1,Wraith King,0.0,0.0,0.0,175.0,2.0,124.0,526.0,...,0.0,0.0,0.0,1.0,1.0,1,1,0.0,0.0,0.0
2,0,1,Riki,0.0,0.0,0.0,137.0,1.0,93.0,439.0,...,0.0,0.0,1.0,1.0,1.0,1,2,0.0,0.0,0.0
3,0,1,Tusk,0.0,0.0,0.0,100.0,0.0,62.0,200.0,...,0.0,0.0,0.0,0.0,1.0,1,3,0.0,0.0,0.0
4,0,1,Templar Assassin,0.0,0.0,0.0,320.0,3.0,352.0,668.0,...,0.0,0.0,0.0,1.0,2.0,1,4,0.0,0.0,0.0


In [33]:
dfkek = df.copy()

In [34]:
df = new_df.copy()

In [35]:
df['xp_600'] = df['xp_600']-df['xp_delta']

In [38]:
df.columns

Index(['match_id', 'radiant', 'hero', 'gold_0', 'lh_0', 'xp_0', 'gold_60',
       'lh_60', 'xp_60', 'gold_120', 'lh_120', 'xp_120', 'gold_180', 'lh_180',
       'xp_180', 'gold_240', 'lh_240', 'xp_240', 'gold_300', 'lh_300',
       'xp_300', 'gold_360', 'lh_360', 'xp_360', 'gold_420', 'lh_420',
       'xp_420', 'gold_480', 'lh_480', 'xp_480', 'gold_540', 'lh_540',
       'xp_540', 'gold_600', 'lh_600', 'xp_600', 'level_180', 'level_240',
       'level_300', 'level_360', 'level_420', 'level_480', 'level_540',
       'level_600', 'radiant_win', 'player_slot', 'damage', 'gold_delta',
       'xp_delta'],
      dtype='object')

In [40]:
minutes = list(range(60,601,60))
cols = ['xp_'+str(i) for i in minutes]
cols2 = ['gold_0', 'lh_0', 'xp_0','level_180', 'level_240',
       'level_300', 'level_360', 'level_420', 'level_480', 'level_540',
       'level_600','player_slot']
xps = df.drop(cols2,axis=1)
xps.head()

,match_id,radiant,hero,gold_60,lh_60,xp_60,gold_120,lh_120,xp_120,gold_180,...,gold_540,lh_540,xp_540,gold_600,lh_600,xp_600,radiant_win,damage,gold_delta,xp_delta
0,0,1,Rubick,100.0,0.0,46.0,250.0,1.0,184.0,350.0,...,1469.0,2.0,1571.0,1569.0,2.0,1741.0,1,0.0,0.0,0.0
1,0,1,Wraith King,175.0,2.0,124.0,526.0,8.0,429.0,927.0,...,3099.0,45.0,2149.0,3367.0,49.0,2319.0,1,0.0,0.0,0.0
2,0,1,Riki,137.0,1.0,93.0,439.0,6.0,485.0,620.0,...,2219.0,32.0,3302.0,2489.0,36.0,3859.0,1,0.0,0.0,0.0
3,0,1,Tusk,100.0,0.0,62.0,200.0,0.0,119.0,300.0,...,1972.0,3.0,1603.0,2072.0,3.0,1676.0,1,0.0,0.0,0.0
4,0,1,Templar Assassin,320.0,3.0,352.0,668.0,9.0,744.0,940.0,...,3452.0,54.0,4154.0,3726.0,58.0,4453.0,1,0.0,0.0,0.0


In [41]:
xps['alone1'], xps['alone2'] = np.zeros(len(xps)), np.zeros(len(xps))
xps['carry'], xps['othplayers'] = np.zeros(len(xps)), np.zeros(len(xps))

In [42]:
import time
start_time = time.time()
k = 0 # счетчик по игрокам команды
while k < len(xps)-1:
    leaders = xps.loc[k:k+4,'xp_600'].nlargest(5).index.values
    xps.set_value(leaders[0], 'alone1', 1)
    xps.set_value(leaders[1], 'alone2', 1)
    #carry
    condition = xps.loc[leaders[2:],'lh_600'].idxmax()
    xps.loc[leaders[2],'carry'] =1 if  condition == leaders[2] else 0
    xps.loc[leaders[3],'carry'] =1 if  condition == leaders[3] else 0
    xps.loc[leaders[4],'carry'] =1 if  condition == leaders[4] else 0
    k += 5

print("--- %s seconds ---" % (time.time() - start_time))

--- 1040.3130686283112 seconds ---


In [43]:
# othplayers
xps['othplayers'] = 1 - xps['alone1'] - xps['alone2'] - xps['carry']

In [44]:
xps2 = xps.copy()

In [45]:
xps_dummy = pd.get_dummies(data=xps,columns=['hero'])
xps_dummy.head()

,match_id,radiant,gold_60,lh_60,xp_60,gold_120,lh_120,xp_120,gold_180,lh_180,...,hero_Venomancer,hero_Viper,hero_Visage,hero_Warlock,hero_Weaver,hero_Windranger,hero_Winter Wyvern,hero_Witch Doctor,hero_Wraith King,hero_Zeus
0,0,1,100.0,0.0,46.0,250.0,1.0,184.0,350.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,175.0,2.0,124.0,526.0,8.0,429.0,927.0,15.0,...,0,0,0,0,0,0,0,0,1,0
2,0,1,137.0,1.0,93.0,439.0,6.0,485.0,620.0,8.0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,100.0,0.0,62.0,200.0,0.0,119.0,300.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,320.0,3.0,352.0,668.0,9.0,744.0,940.0,12.0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
index = list(xps_dummy['match_id'].unique())
columns = list(xps_dummy.columns[-110:])

In [47]:
dummy_match = pd.DataFrame(0, index=index,columns=columns)
dummy_match.head()

,hero_Abaddon,hero_Alchemist,hero_Ancient Apparition,hero_Anti-Mage,hero_Axe,hero_Bane,hero_Batrider,hero_Beastmaster,hero_Bloodseeker,hero_Bounty Hunter,...,hero_Venomancer,hero_Viper,hero_Visage,hero_Warlock,hero_Weaver,hero_Windranger,hero_Winter Wyvern,hero_Witch Doctor,hero_Wraith King,hero_Zeus
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
j = 0
for i in range(len(xps_dummy)):
    if i % 10 < 5:
        dummy_match.loc[j,:] += xps_dummy.loc[i,columns]
    else:
        dummy_match.loc[j,:] -= xps_dummy.loc[i,columns]
    if i % 10 == 9:
        j += 1

In [49]:
dummy_match.head(10)

,hero_Abaddon,hero_Alchemist,hero_Ancient Apparition,hero_Anti-Mage,hero_Axe,hero_Bane,hero_Batrider,hero_Beastmaster,hero_Bloodseeker,hero_Bounty Hunter,...,hero_Venomancer,hero_Viper,hero_Visage,hero_Warlock,hero_Weaver,hero_Windranger,hero_Winter Wyvern,hero_Witch Doctor,hero_Wraith King,hero_Zeus
0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-1.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0
6,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,-1.0,0.0,1.0,0.0


In [50]:
xps = xps2.copy()
xps["Num"] = xps.groupby('match_id')['match_id'].transform(lambda x: pd.Series(range(1, len(x)+1), index=x.index))
xps.head(11)

,match_id,radiant,hero,gold_60,lh_60,xp_60,gold_120,lh_120,xp_120,gold_180,...,xp_600,radiant_win,damage,gold_delta,xp_delta,alone1,alone2,carry,othplayers,Num
0,0,1,Rubick,100.0,0.0,46.0,250.0,1.0,184.0,350.0,...,1741.0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,0,1,Wraith King,175.0,2.0,124.0,526.0,8.0,429.0,927.0,...,2319.0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
2,0,1,Riki,137.0,1.0,93.0,439.0,6.0,485.0,620.0,...,3859.0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3
3,0,1,Tusk,100.0,0.0,62.0,200.0,0.0,119.0,300.0,...,1676.0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
4,0,1,Templar Assassin,320.0,3.0,352.0,668.0,9.0,744.0,940.0,...,4453.0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5
5,0,0,Bounty Hunter,100.0,0.0,72.0,200.0,0.0,169.0,450.0,...,1644.0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6
6,0,0,Invoker,143.0,1.0,155.0,362.0,4.0,479.0,651.0,...,2653.0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7
7,0,0,Earthshaker,100.0,0.0,103.0,255.0,2.0,350.0,505.0,...,1804.0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8
8,0,0,Anti-Mage,178.0,2.0,103.0,522.0,8.0,392.0,1102.0,...,3011.0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,9
9,0,0,Shadow Fiend,242.0,1.0,255.0,508.0,5.0,719.0,879.0,...,2965.0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10


In [51]:
xps3 = xps.copy()

In [52]:
xps = xps.drop(["radiant"], axis=1)

In [53]:
xps = xps.groupby(['match_id', "Num"]).sum().unstack()
xps = xps.sort_index(axis=1, level=1)
xps.columns = ['_'.join((col[0], str(col[1]))) for col in xps.columns]
xps.columns

Index(['gold_60_1', 'lh_60_1', 'xp_60_1', 'gold_120_1', 'lh_120_1', 'xp_120_1',
       'gold_180_1', 'lh_180_1', 'xp_180_1', 'gold_240_1',
       ...
       'lh_600_10', 'xp_600_10', 'radiant_win_10', 'damage_10',
       'gold_delta_10', 'xp_delta_10', 'alone1_10', 'alone2_10', 'carry_10',
       'othplayers_10'],
      dtype='object', length=380)

In [54]:
xps = xps.drop(["radiant_win_10", "radiant_win_2", "radiant_win_3", "radiant_win_4", "radiant_win_5", "radiant_win_6", "radiant_win_7", "radiant_win_8", "radiant_win_9"], axis=1)

In [55]:
xps4 = xps.copy()

In [19]:
xps = xps4.copy() # для удобства делать бэкап

In [56]:
list(xps4.columns) # потом с помощью этого списка можно будет дропнуть переменные

['gold_60_1',
 'lh_60_1',
 'xp_60_1',
 'gold_120_1',
 'lh_120_1',
 'xp_120_1',
 'gold_180_1',
 'lh_180_1',
 'xp_180_1',
 'gold_240_1',
 'lh_240_1',
 'xp_240_1',
 'gold_300_1',
 'lh_300_1',
 'xp_300_1',
 'gold_360_1',
 'lh_360_1',
 'xp_360_1',
 'gold_420_1',
 'lh_420_1',
 'xp_420_1',
 'gold_480_1',
 'lh_480_1',
 'xp_480_1',
 'gold_540_1',
 'lh_540_1',
 'xp_540_1',
 'gold_600_1',
 'lh_600_1',
 'xp_600_1',
 'radiant_win_1',
 'damage_1',
 'gold_delta_1',
 'xp_delta_1',
 'alone1_1',
 'alone2_1',
 'carry_1',
 'othplayers_1',
 'gold_60_2',
 'lh_60_2',
 'xp_60_2',
 'gold_120_2',
 'lh_120_2',
 'xp_120_2',
 'gold_180_2',
 'lh_180_2',
 'xp_180_2',
 'gold_240_2',
 'lh_240_2',
 'xp_240_2',
 'gold_300_2',
 'lh_300_2',
 'xp_300_2',
 'gold_360_2',
 'lh_360_2',
 'xp_360_2',
 'gold_420_2',
 'lh_420_2',
 'xp_420_2',
 'gold_480_2',
 'lh_480_2',
 'xp_480_2',
 'gold_540_2',
 'lh_540_2',
 'xp_540_2',
 'gold_600_2',
 'lh_600_2',
 'xp_600_2',
 'damage_2',
 'gold_delta_2',
 'xp_delta_2',
 'alone1_2',
 'alone2_2

In [57]:
xps.head()

,gold_60_1,lh_60_1,xp_60_1,gold_120_1,lh_120_1,xp_120_1,gold_180_1,lh_180_1,xp_180_1,gold_240_1,...,gold_600_10,lh_600_10,xp_600_10,damage_10,gold_delta_10,xp_delta_10,alone1_10,alone2_10,carry_10,othplayers_10
match_id,,,,,,,,,,,,,,,,,,,,,
0,100.0,0.0,46.0,250.0,1.0,184.0,350.0,1.0,323.0,450.0,...,2804.0,38.0,2965.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,263.0,4.0,289.0,480.0,7.0,639.0,741.0,11.0,928.0,1130.0,...,2375.0,25.0,2186.0,359.0,172.0,383.0,0.0,0.0,1.0,0.0
2,565.0,3.0,339.0,1264.0,11.0,1078.0,1675.0,11.0,1321.0,1979.0,...,3542.0,25.0,2477.0,220.0,-65.0,20.0,0.0,1.0,0.0,0.0
3,217.0,3.0,93.0,512.0,8.0,339.0,698.0,10.0,565.0,1261.0,...,2216.0,18.0,2699.0,317.0,-117.0,562.0,0.0,1.0,0.0,0.0
4,100.0,0.0,113.0,200.0,0.0,303.0,300.0,0.0,513.0,400.0,...,1686.0,1.0,1968.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [58]:
def roles(cname,sec):
    xps[cname+str(sec)+'_alone1_R'], xps[cname+str(sec)+'_alone1_D'] = np.zeros(len(xps)), np.zeros(len(xps))
    xps[cname+str(sec)+'_alone2_R'], xps[cname+str(sec)+'_alone2_D'] = np.zeros(len(xps)), np.zeros(len(xps))
    xps[cname+str(sec)+'_carry_R'], xps[cname+str(sec)+'_carry_D'] = np.zeros(len(xps)), np.zeros(len(xps))
    xps[cname+str(sec)+'_others_R'], xps[cname+str(sec)+'_others_D'] = np.zeros(len(xps)), np.zeros(len(xps))
    
    # сумма по признаку на любую минуту для 4-х ролей игроков команды светлых
    for i in range(1,6):
        xps[cname+str(sec)+'_alone1_R'] += xps[cname+'_'+str(sec)+'_'+str(i)] * xps['alone1_'+str(i)]
        xps[cname+str(sec)+'_alone2_R'] += xps[cname+'_'+str(sec)+'_'+str(i)] * xps['alone2_'+str(i)]
        xps[cname+str(sec)+'_carry_R'] += xps[cname+'_'+str(sec)+'_'+str(i)] * xps['carry_'+str(i)]
        xps[cname+str(sec)+'_others_R'] += xps[cname+'_'+str(sec)+'_'+str(i)] * xps['othplayers_'+str(i)]
    # сумма по признаку на любую минуту для 4-х ролей игроков команды темных
    for i in range(6,11):
        xps[cname+str(sec)+'_alone1_D'] -= xps[cname+'_'+str(sec)+'_'+str(i)] * xps['alone1_'+str(i)]
        xps[cname+str(sec)+'_alone2_D'] -= xps[cname+'_'+str(sec)+'_'+str(i)] * xps['alone2_'+str(i)]
        xps[cname+str(sec)+'_carry_D'] -= xps[cname+'_'+str(sec)+'_'+str(i)] * xps['carry_'+str(i)]
        xps[cname+str(sec)+'_others_D'] -= xps[cname+'_'+str(sec)+'_'+str(i)] * xps['othplayers_'+str(i)]

In [59]:
roles('gold',600)
roles('xp',600)
roles('lh',600)

In [60]:
roles('gold',300)
roles('xp',300)
roles('lh',300)

In [61]:
xps.head(10)

,gold_60_1,lh_60_1,xp_60_1,gold_120_1,lh_120_1,xp_120_1,gold_180_1,lh_180_1,xp_180_1,gold_240_1,...,xp300_others_R,xp300_others_D,lh300_alone1_R,lh300_alone1_D,lh300_alone2_R,lh300_alone2_D,lh300_carry_R,lh300_carry_D,lh300_others_R,lh300_others_D
match_id,,,,,,,,,,,,,,,,,,,,,
0,100.0,0.0,46.0,250.0,1.0,184.0,350.0,1.0,323.0,450.0,...,1433.0,-1467.0,26.0,-30.0,17.0,-14.0,24.0,-10.0,4.0,-4.0
1,263.0,4.0,289.0,480.0,7.0,639.0,741.0,11.0,928.0,1130.0,...,2014.0,-1919.0,21.0,-26.0,26.0,-5.0,20.0,-10.0,5.0,-9.0
2,565.0,3.0,339.0,1264.0,11.0,1078.0,1675.0,11.0,1321.0,1979.0,...,1763.0,-1614.0,19.0,-10.0,29.0,-14.0,10.0,-2.0,2.0,-5.0
3,217.0,3.0,93.0,512.0,8.0,339.0,698.0,10.0,565.0,1261.0,...,1837.0,-1926.0,18.0,-15.0,2.0,-12.0,18.0,-28.0,22.0,-21.0
4,100.0,0.0,113.0,200.0,0.0,303.0,300.0,0.0,513.0,400.0,...,2032.0,-1512.0,17.0,-20.0,4.0,-24.0,19.0,-10.0,20.0,-7.0
5,99.0,0.0,20.0,199.0,0.0,20.0,431.0,2.0,256.0,557.0,...,1614.0,-1378.0,30.0,-19.0,18.0,-27.0,4.0,-4.0,5.0,-7.0
6,99.0,0.0,0.0,249.0,1.0,123.0,349.0,1.0,183.0,843.0,...,1603.0,-1117.0,33.0,-16.0,13.0,-27.0,4.0,-6.0,3.0,-1.0
7,100.0,0.0,82.0,200.0,0.0,277.0,332.0,1.0,385.0,584.0,...,1738.0,-1925.0,19.0,-24.0,9.0,-11.0,8.0,-15.0,9.0,-20.0
8,200.0,0.0,389.0,380.0,2.0,750.0,615.0,5.0,1142.0,1028.0,...,1382.0,-1608.0,13.0,-10.0,0.0,-9.0,8.0,-9.0,8.0,-8.0


In [62]:
itog = pd.concat([dummy_match, xps], axis=1)
itog.head()

,hero_Abaddon,hero_Alchemist,hero_Ancient Apparition,hero_Anti-Mage,hero_Axe,hero_Bane,hero_Batrider,hero_Beastmaster,hero_Bloodseeker,hero_Bounty Hunter,...,xp300_others_R,xp300_others_D,lh300_alone1_R,lh300_alone1_D,lh300_alone2_R,lh300_alone2_D,lh300_carry_R,lh300_carry_D,lh300_others_R,lh300_others_D
match_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,1433.0,-1467.0,26.0,-30.0,17.0,-14.0,24.0,-10.0,4.0,-4.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2014.0,-1919.0,21.0,-26.0,26.0,-5.0,20.0,-10.0,5.0,-9.0
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-1.0,-1.0,0.0,...,1763.0,-1614.0,19.0,-10.0,29.0,-14.0,10.0,-2.0,2.0,-5.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1837.0,-1926.0,18.0,-15.0,2.0,-12.0,18.0,-28.0,22.0,-21.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2032.0,-1512.0,17.0,-20.0,4.0,-24.0,19.0,-10.0,20.0,-7.0


In [63]:
sas = list(xps4.columns)
sas.remove('radiant_win_1')

In [64]:
data = itog.drop(sas,axis=1)

In [65]:
data.columns

Index(['hero_Abaddon', 'hero_Alchemist', 'hero_Ancient Apparition',
       'hero_Anti-Mage', 'hero_Axe', 'hero_Bane', 'hero_Batrider',
       'hero_Beastmaster', 'hero_Bloodseeker', 'hero_Bounty Hunter',
       ...
       'xp300_others_R', 'xp300_others_D', 'lh300_alone1_R', 'lh300_alone1_D',
       'lh300_alone2_R', 'lh300_alone2_D', 'lh300_carry_R', 'lh300_carry_D',
       'lh300_others_R', 'lh300_others_D'],
      dtype='object', length=159)

In [51]:
data.shape

(39959, 159)

In [66]:
y = data['radiant_win_1']
X = data.drop('radiant_win_1',axis=1)

In [67]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [69]:
parameters = {'penalty':('l1', 'l2'), 'C':[.1,.5,.7,1.,2.]}
logist = LogisticRegression(random_state=42)
regr = GridSearchCV(logist, parameters,cv=5,scoring='roc_auc')
regr.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ('l1', 'l2'), 'C': [0.1, 0.5, 0.7, 1.0, 2.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [70]:
regr.best_estimator_

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [71]:
regr.best_score_

0.75796503387748537

In [72]:
regr.best_params_

{'C': 0.5, 'penalty': 'l1'}

In [73]:
# тестим наш грид серч
from sklearn.metrics import roc_auc_score

In [74]:
mymodel = LogisticRegression(C=regr.best_params_['C'],penalty=regr.best_params_['penalty'],random_state=42)
mymodel.fit(X_train,y_train)

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [75]:
predictions = mymodel.predict(X_train)

In [76]:
roc_auc_score(predictions, y_train)

0.69343258334477353

In [77]:
predictions = mymodel.predict(X_test)#отложенная обучающая выборка

In [78]:
roc_auc_score(predictions, y_test)

0.69286872052685677

In [79]:
mymodel.fit(X,y)#вся обучающая выборка

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Загружаем тестовую выборку

In [125]:
df_test = pd.read_csv('main_test.csv')
tf_test = pd.read_csv('teamfights_test.csv')

In [126]:
df_test.head()

,match_id,radiant,hero,gold_0,lh_0,xp_0,gold_60,lh_60,xp_60,gold_120,...,lh_600,xp_600,level_180,level_240,level_300,level_360,level_420,level_480,level_540,level_600
0,39959,1,Ancient Apparition,0.0,0.0,0.0,100.0,0.0,0.0,809.0,...,4.0,1492.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,3.0
1,39959,1,Vengeful Spirit,0.0,0.0,0.0,100.0,0.0,124.0,200.0,...,5.0,2034.0,0.0,0.0,1.0,1.0,1.0,2.0,2.0,3.0
2,39959,1,Shadow Fiend,0.0,0.0,0.0,141.0,1.0,206.0,363.0,...,33.0,3560.0,0.0,0.0,1.0,1.0,1.0,3.0,4.0,5.0
3,39959,1,Sven,0.0,0.0,0.0,218.0,3.0,155.0,672.0,...,26.0,2548.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,4.0
4,39959,1,Anti-Mage,0.0,0.0,0.0,141.0,1.0,93.0,319.0,...,37.0,3435.0,0.0,0.0,1.0,1.0,1.0,2.0,2.0,4.0


In [127]:
len(df_test) / 10

9989.0

In [128]:
x = [0,1,2,3,4,128,129,130,131,132]
df_test['player_slot'] = x * 9989

In [129]:
tf1 = tf_test.copy()
tf1['xp_delta'] = tf1['xp_end']-tf1['xp_start']
tf1['time'] = np.ceil(tf1['end'] / 60) * 60
tf2 = tf1[['teamfight_id','match_id','player_slot','radiant','damage','gold_delta','xp_delta','time']]

In [130]:
tf3 = tf2.groupby(['match_id','player_slot'],as_index=False)[['damage','gold_delta','xp_delta']].sum()

In [131]:
tf3.columns

Index(['match_id', 'player_slot', 'damage', 'gold_delta', 'xp_delta'], dtype='object')

In [132]:
len(tf3)

63320

In [133]:
new_df = pd.merge(df_test, tf3,  how='left', left_on=['match_id','player_slot'], right_on = ['match_id','player_slot'])
new_df = new_df.fillna(0)

In [134]:
len(new_df)

99890

In [135]:
df_test = new_df.copy()

In [136]:
df_test['xp_600'] = df_test['xp_600']-df_test['xp_delta']

In [137]:
minutes = list(range(60,601,60))
cols = ['xp_'+str(i) for i in minutes]
cols2 = ['gold_0', 'lh_0', 'xp_0','level_180', 'level_240',
       'level_300', 'level_360', 'level_420', 'level_480', 'level_540',
       'level_600','player_slot']
xps = df_test.drop(cols2,axis=1)
xps.head()

,match_id,radiant,hero,gold_60,lh_60,xp_60,gold_120,lh_120,xp_120,gold_180,...,xp_480,gold_540,lh_540,xp_540,gold_600,lh_600,xp_600,damage,gold_delta,xp_delta
0,39959,1,Ancient Apparition,100.0,0.0,0.0,809.0,1.0,324.0,909.0,...,1166.0,1927.0,4.0,1390.0,2027.0,4.0,1029.0,588.0,609.0,463.0
1,39959,1,Vengeful Spirit,100.0,0.0,124.0,200.0,0.0,257.0,337.0,...,1726.0,1604.0,5.0,1757.0,1704.0,5.0,1910.0,303.0,42.0,124.0
2,39959,1,Shadow Fiend,141.0,1.0,206.0,363.0,4.0,546.0,687.0,...,2886.0,3011.0,29.0,3407.0,3204.0,33.0,3106.0,924.0,411.0,454.0
3,39959,1,Sven,218.0,3.0,155.0,672.0,4.0,654.0,853.0,...,2053.0,2174.0,21.0,2146.0,2482.0,26.0,1915.0,648.0,216.0,633.0
4,39959,1,Anti-Mage,141.0,1.0,93.0,319.0,3.0,226.0,629.0,...,2750.0,2903.0,32.0,3158.0,3215.0,37.0,3084.0,342.0,118.0,351.0


In [138]:
len(xps)

99890

In [139]:
xps['alone1'], xps['alone2'] = np.zeros(len(xps)), np.zeros(len(xps))
xps['carry'], xps['othplayers'] = np.zeros(len(xps)), np.zeros(len(xps))

In [140]:
import time
start_time = time.time()
k = 0 # счетчик по игрокам команды
while k < len(xps)-1:
    leaders = xps.loc[k:k+4,'xp_600'].nlargest(5).index.values
    xps.set_value(leaders[0], 'alone1', 1)
    xps.set_value(leaders[1], 'alone2', 1)
    #carry
    condition = xps.loc[leaders[2:],'lh_600'].idxmax()
    xps.loc[leaders[2],'carry'] =1 if  condition == leaders[2] else 0
    xps.loc[leaders[3],'carry'] =1 if  condition == leaders[3] else 0
    xps.loc[leaders[4],'carry'] =1 if  condition == leaders[4] else 0
    k += 5

print("--- %s seconds ---" % (time.time() - start_time))

--- 64.78740930557251 seconds ---


In [141]:
# othplayers
xps['othplayers'] = 1 - xps['alone1'] - xps['alone2'] - xps['carry']

In [142]:
xps2=xps.copy()

In [143]:
len(xps2)

99890

In [144]:
xps_dummy = pd.get_dummies(data=xps,columns=['hero'])
xps_dummy.head()

,match_id,radiant,gold_60,lh_60,xp_60,gold_120,lh_120,xp_120,gold_180,lh_180,...,hero_Venomancer,hero_Viper,hero_Visage,hero_Warlock,hero_Weaver,hero_Windranger,hero_Winter Wyvern,hero_Witch Doctor,hero_Wraith King,hero_Zeus
0,39959,1,100.0,0.0,0.0,809.0,1.0,324.0,909.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,39959,1,100.0,0.0,124.0,200.0,0.0,257.0,337.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,39959,1,141.0,1.0,206.0,363.0,4.0,546.0,687.0,8.0,...,0,0,0,0,0,0,0,0,0,0
3,39959,1,218.0,3.0,155.0,672.0,4.0,654.0,853.0,6.0,...,0,0,0,0,0,0,0,0,0,0
4,39959,1,141.0,1.0,93.0,319.0,3.0,226.0,629.0,8.0,...,0,0,0,0,0,0,0,0,0,0


In [145]:
index = list(xps_dummy['match_id'].unique())
columns = list(xps_dummy.columns[-110:])

In [146]:
dummy_match = pd.DataFrame(0, index=index,columns=columns)
dummy_match.head()

,hero_Abaddon,hero_Alchemist,hero_Ancient Apparition,hero_Anti-Mage,hero_Axe,hero_Bane,hero_Batrider,hero_Beastmaster,hero_Bloodseeker,hero_Bounty Hunter,...,hero_Venomancer,hero_Viper,hero_Visage,hero_Warlock,hero_Weaver,hero_Windranger,hero_Winter Wyvern,hero_Witch Doctor,hero_Wraith King,hero_Zeus
39959,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39960,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39961,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39962,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39963,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [148]:
j=39959
for i in range(len(xps_dummy)):
    if i % 10 < 5:
        dummy_match.loc[j,:] += xps_dummy.loc[i,columns]
    else:
        dummy_match.loc[j,:] -= xps_dummy.loc[i,columns]
    if i % 10 == 9:
        j += 1

In [149]:
dummy_match.shape

(9989, 110)

In [150]:
xps = xps2.copy()
xps["Num"] = xps.groupby('match_id')['match_id'].transform(lambda x: pd.Series(range(1, len(x)+1), index=x.index))
xps.head(11)

,match_id,radiant,hero,gold_60,lh_60,xp_60,gold_120,lh_120,xp_120,gold_180,...,lh_600,xp_600,damage,gold_delta,xp_delta,alone1,alone2,carry,othplayers,Num
0,39959,1,Ancient Apparition,100.0,0.0,0.0,809.0,1.0,324.0,909.0,...,4.0,1029.0,588.0,609.0,463.0,0.0,0.0,0.0,1.0,1
1,39959,1,Vengeful Spirit,100.0,0.0,124.0,200.0,0.0,257.0,337.0,...,5.0,1910.0,303.0,42.0,124.0,0.0,0.0,0.0,1.0,2
2,39959,1,Shadow Fiend,141.0,1.0,206.0,363.0,4.0,546.0,687.0,...,33.0,3106.0,924.0,411.0,454.0,1.0,0.0,0.0,0.0,3
3,39959,1,Sven,218.0,3.0,155.0,672.0,4.0,654.0,853.0,...,26.0,1915.0,648.0,216.0,633.0,0.0,0.0,1.0,0.0,4
4,39959,1,Anti-Mage,141.0,1.0,93.0,319.0,3.0,226.0,629.0,...,37.0,3084.0,342.0,118.0,351.0,0.0,1.0,0.0,0.0,5
5,39959,0,Windranger,291.0,2.0,213.0,432.0,3.0,326.0,754.0,...,24.0,2335.0,864.0,396.0,595.0,0.0,1.0,0.0,0.0,6
6,39959,0,Lina,100.0,0.0,93.0,237.0,1.0,392.0,376.0,...,6.0,1682.0,134.0,-112.0,299.0,0.0,0.0,0.0,1.0,7
7,39959,0,Pudge,141.0,1.0,113.0,284.0,2.0,288.0,424.0,...,10.0,1141.0,627.0,-135.0,124.0,0.0,0.0,0.0,1.0,8
8,39959,0,Queen of Pain,220.0,3.0,258.0,713.0,5.0,798.0,941.0,...,36.0,3327.0,383.0,520.0,674.0,1.0,0.0,0.0,0.0,9
9,39959,0,Wraith King,361.0,4.0,234.0,555.0,8.0,460.0,770.0,...,28.0,1405.0,356.0,94.0,355.0,0.0,0.0,1.0,0.0,10


In [151]:
xps = xps.drop(["radiant"], axis=1)

In [152]:
xps = xps.groupby(['match_id', "Num"]).sum().unstack()
xps = xps.sort_index(axis=1, level=1)
xps.columns = ['_'.join((col[0], str(col[1]))) for col in xps.columns]
xps.columns

Index(['gold_60_1', 'lh_60_1', 'xp_60_1', 'gold_120_1', 'lh_120_1', 'xp_120_1',
       'gold_180_1', 'lh_180_1', 'xp_180_1', 'gold_240_1',
       ...
       'gold_600_10', 'lh_600_10', 'xp_600_10', 'damage_10', 'gold_delta_10',
       'xp_delta_10', 'alone1_10', 'alone2_10', 'carry_10', 'othplayers_10'],
      dtype='object', length=370)

In [113]:
#xps = xps.drop(["radiant_win_10", "radiant_win_2", "radiant_win_3", "radiant_win_4", "radiant_win_5", "radiant_win_6", "radiant_win_7", "radiant_win_8", "radiant_win_9"], axis=1)

In [153]:
xps4 = xps.copy()

In [154]:
roles('gold',600)
roles('xp',600)
roles('lh',600)

In [155]:
roles('gold',300)
roles('xp',300)
roles('lh',300)

In [156]:
itog1 = pd.concat([dummy_match, xps], axis=1)
itog1.head()

,hero_Abaddon,hero_Alchemist,hero_Ancient Apparition,hero_Anti-Mage,hero_Axe,hero_Bane,hero_Batrider,hero_Beastmaster,hero_Bloodseeker,hero_Bounty Hunter,...,xp300_others_R,xp300_others_D,lh300_alone1_R,lh300_alone1_D,lh300_alone2_R,lh300_alone2_D,lh300_carry_R,lh300_carry_D,lh300_others_R,lh300_others_D
match_id,,,,,,,,,,,,,,,,,,,,,
39959,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2090.0,-2064.0,18.0,-16.0,17.0,-14.0,10.0,-17.0,3.0,-11.0
39960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1313.0,-963.0,25.0,-11.0,18.0,-16.0,24.0,-8.0,1.0,-3.0
39961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,710.0,-1301.0,18.0,-22.0,26.0,-8.0,5.0,-16.0,4.0,-3.0
39962,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1464.0,-1736.0,18.0,-17.0,18.0,-9.0,19.0,-4.0,2.0,-11.0
39963,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1125.0,-1482.0,15.0,-10.0,16.0,-21.0,23.0,-19.0,5.0,-4.0


In [157]:
sas = list(xps4.columns)
#sas.remove('radiant_win_1')

In [158]:
data_test = itog1.drop(sas,axis=1)

In [159]:
data_test.shape

(9989, 158)

In [160]:
#y = data_test['radiant_win_1']
X = data_test
X.head()

,hero_Abaddon,hero_Alchemist,hero_Ancient Apparition,hero_Anti-Mage,hero_Axe,hero_Bane,hero_Batrider,hero_Beastmaster,hero_Bloodseeker,hero_Bounty Hunter,...,xp300_others_R,xp300_others_D,lh300_alone1_R,lh300_alone1_D,lh300_alone2_R,lh300_alone2_D,lh300_carry_R,lh300_carry_D,lh300_others_R,lh300_others_D
match_id,,,,,,,,,,,,,,,,,,,,,
39959,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2090.0,-2064.0,18.0,-16.0,17.0,-14.0,10.0,-17.0,3.0,-11.0
39960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1313.0,-963.0,25.0,-11.0,18.0,-16.0,24.0,-8.0,1.0,-3.0
39961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,710.0,-1301.0,18.0,-22.0,26.0,-8.0,5.0,-16.0,4.0,-3.0
39962,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1464.0,-1736.0,18.0,-17.0,18.0,-9.0,19.0,-4.0,2.0,-11.0
39963,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1125.0,-1482.0,15.0,-10.0,16.0,-21.0,23.0,-19.0,5.0,-4.0


In [161]:
predictions_test = mymodel.predict_proba(X)

In [162]:
indeces = np.array(df_test.match_id.values[::10],dtype=int)
submission = pd.DataFrame()
submission['index'] = indeces
submission['proba'] = predictions_test[:,1]

In [163]:
submission.to_csv('attempt2.csv',index=False)

In [164]:
len(submission)

9989

In [165]:
submission.head()

,index,proba
0,39959,0.804307
1,39960,0.895492
2,39961,0.294948
3,39962,0.554803
4,39963,0.310719
